In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa
import cv2
from ultralytics import YOLO
import tensorflow as tf
import gc

In [2]:
def get_unet_mask(image_path, Unet):
    """
    Genera una máscara de segmentación utilizando un modelo Unet.

    Parámetros:
    image_path (str): Ruta de la imagen de entrada.
    Unet (modelo): Modelo Unet preentrenado.

    Retorna:
    numpy.ndarray: Máscara de segmentación binaria.
    """
    image = cv2.imread(image_path)
    img_orig = image.astype(np.float32)
    img_orig_gray = cv2.cvtColor(img_orig, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img_orig_gray, (256, 512))
    normalizedImg = np.zeros(img.shape)
    img = cv2.normalize(img, normalizedImg, -1, 1, cv2.NORM_MINMAX)
    img = img[None, ..., None]

    pred_maps, seg_pred = Unet.predict(img, verbose=0)
    mask = np.asarray(np.squeeze(seg_pred))
    mask = np.round(mask)

    mask = cv2.resize(mask, (img_orig_gray.shape[1], img_orig_gray.shape[0]))
    mask = mask.astype(bool)
    return mask

In [ ]:
def get_yolo_mask(image_path, model):
    """
    Genera una máscara de segmentación utilizando un modelo YOLO.

    Parámetros:
    image_path (str): Ruta de la imagen de entrada.
    model (YOLO): Modelo YOLO preentrenado.

    Retorna:
    numpy.ndarray: Máscara de segmentación binaria.
    """
    image = cv2.imread(image_path)
    results = model(image)
    mask = np.zeros(image.shape[:2], dtype=np.uint8)

    if results[0].masks is not None:
        masks = results[0].masks.xy
        for mask_array in masks:
            mask_array = mask_array.astype(np.int32)
            cv2.fillPoly(mask, [mask_array], 1)
            
    mask = mask.astype(bool)
    return mask


In [4]:
def calculate_iou(mask1, mask2):
    """
    Calcula el Índice de Superposición (IoU) entre dos máscaras binarias.

    Parámetros:
    mask1 (numpy.ndarray): Primera máscara binaria.
    mask2 (numpy.ndarray): Segunda máscara binaria.

    Retorna:
    float: Valor del IoU.
    """
    if np.sum(mask1) == 0 and np.sum(mask2) == 0:
        return 1.0  # Ambas máscaras son cero, IoU es 1
    elif np.sum(mask1) == 0 or np.sum(mask2) == 0:
        return 0.0  # Una de las máscaras es cero, IoU es 0
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou = np.sum(intersection) / np.sum(union)
    return iou


In [11]:
# Rutas de los archivos de entrada y modelos
image_path = "/home/voicelab/Desktop/segmentation_glottis/BAGLS/test/test/0.png"
mask_test_path = "/home/voicelab/Desktop/segmentation_glottis/BAGLS/test/test/0_seg.png"
unet_model_path = "/home/voicelab/Downloads/epoch025.h5"
yolo_model_path = "/home/voicelab/Desktop/segmentation_glottis/YOLOV8/runs/segment/train2/weights/best.pt"

# Leer la máscara de prueba
mask_test = cv2.imread(mask_test_path, cv2.IMREAD_GRAYSCALE)
mask_test = mask_test.astype(bool)

# Cargar modelos
yolo_model_path = YOLO(yolo_model_path)
unet_model_path = load_model(unet_model_path, compile=False, custom_objects={'InstanceNormalization': tfa.layers.InstanceNormalization})

# Obtener la mascara con ambos modelos
unet_mask = get_unet_mask(image_path, unet_model_path)
yolo_mask = get_yolo_mask(image_path, yolo_model_path)

# Calculo de iou en la prediccion de cada modelo
iou_yolo = calculate_iou(mask_test, yolo_mask)
unet_yolo = calculate_iou(mask_test, unet_mask)

print(f"YOLO IoU: {iou_yolo}")
print(f"Unet IoU: {unet_yolo}")

1/1 [==============================] - 0s 367ms/step

0: 640x640 1 glottis, 1.9ms
Speed: 0.9ms preprocess, 1.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
YOLO IoU: 0.7184466019417476
Unet IoU: 0.8394160583941606


In [2]:

# Verificar los dispositivos disponibles
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPU is available and being used.")
else:
    print("GPU is not available, using CPU instead.")

def evaluate_model_unet(test_dir, unet_model_path, batch_size=10):
    """
    Evalúa el modelo Unet en un conjunto de datos de prueba y calcula el IoU promedio.

    Parámetros:
    test_dir (str): Directorio que contiene las imágenes de prueba y sus máscaras correspondientes.
    unet_model_path (str): Ruta del modelo Unet preentrenado.
    batch_size (int): Número de imágenes a procesar en cada lote.

    Retorna:
    None
    """
    unet_model = load_model(unet_model_path, compile=False, custom_objects={'InstanceNormalization': tfa.layers.InstanceNormalization})
    
    unet_ious = []
    images = []
    masks = []

    # Se leen los archivos del conjunto test
    for file_name in os.listdir(test_dir):
        if file_name.endswith('_seg.png'):
            image_name = file_name.replace('_seg.png', '.png')
            mask_path = os.path.join(test_dir, file_name)
            image_path = os.path.join(test_dir, image_name)

            # Leer la mascara del conjnto test
            ground_truth_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE).astype(bool)
            masks.append(ground_truth_mask)
            images.append(image_path)

            if len(images) == batch_size:
                for i, img_path in enumerate(images):
                    unet_mask = get_unet_mask(img_path, unet_model) # Obtener la máscara predicha por el modelo Unet

                    iou_unet = calculate_iou(masks[i], unet_mask) # Calcular el IoU entre la máscara verdadera y la máscara predicha
                    if np.isnan(iou_unet):
                        print(f"Warning: IoU is NaN for image {img_path}")
                    else:
                        unet_ious.append(iou_unet)


                images = []
                masks = []
                gc.collect()

    avg_iou_unet = np.mean(unet_ious)

    print(f"Average Unet IoU: {avg_iou_unet}")

test_dir = "/home/voicelab/Desktop/segmentation_glottis/BAGLS/test/test"
unet_model_path = "/home/voicelab/Downloads/epoch025.h5"

evaluate_model_unet(test_dir, unet_model_path)


GPU is available and being used.
Average Unet IoU: 0.7454116614950004


In [ ]:

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPU is available and being used.")
else:
    print("GPU is not available, using CPU instead.")

def evaluate_model_yolo(test_dir, yolo_model_path, batch_size=5):
    """
    Evalúa el modelo YOLO en un conjunto de datos de prueba y calcula el IoU promedio.

    Parámetros:
    test_dir (str): Directorio que contiene las imágenes de prueba y sus máscaras correspondientes.
    yolo_model_path (str): Ruta del modelo YOLO preentrenado.
    batch_size (int): Número de imágenes a procesar en cada lote.

    Retorna:
    None
    """
    yolo_model = YOLO(yolo_model_path)
    
    yolo_ious = []
    images = []
    masks = []

    for file_name in os.listdir(test_dir):
        if file_name.endswith('_seg.png'):
            image_name = file_name.replace('_seg.png', '.png')
            mask_path = os.path.join(test_dir, file_name)
            image_path = os.path.join(test_dir, image_name)

            ground_truth_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE).astype(bool)
            masks.append(ground_truth_mask)
            images.append(image_path)

            if len(images) == batch_size:
                for i, img_path in enumerate(images):
                    yolo_mask = get_yolo_mask(img_path, yolo_model)

                    iou_yolo = calculate_iou(masks[i], yolo_mask)

                    if np.isnan(iou_yolo):
                        print(f"Warning: IoU is NaN for image {img_path}")
                    else:
                        yolo_ious.append(iou_yolo)

                images = []
                masks = []
                gc.collect()

    avg_iou_yolo = np.mean(yolo_ious)
    print(f"Average YOLO IoU: {avg_iou_yolo}")

test_dir = "/home/voicelab/Desktop/segmentation_glottis/BAGLS/test/test"
yolo_model_path = "/home/voicelab/Desktop/segmentation_glottis/YOLOV8/runs/segment/train2/weights/best.pt"

evaluate_model_yolo(test_dir, yolo_model_path)


2024-06-19 02:23:24.382772: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-19 02:23:24.403208: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-19 02:23:24.403225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-19 02:23:24.403808: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-19 02:23:24.407773: I tensorflow/core/platform/cpu_feature_guar

GPU is available and being used.



2024-06-19 02:23:26.199734: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-19 02:23:26.200453: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-19 02:23:26.200525: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

0: 640x512 1 glottis, 58.1ms
Speed: 2.8ms preprocess, 58.1ms inference, 59.9ms postprocess per image at shape (1, 3, 640, 512)

0: 640x288 1 glottis, 57.4ms
Speed: 0.5ms preprocess, 57.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 288)

0: 640x640 (no detections), 2.2ms
Speed: 1.1ms preprocess, 2.2ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 glottis, 2.2ms
Speed: 1.1ms preprocess, 2.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x384 1 glottis, 55.9ms
Speed: 0.7ms preprocess, 55.9ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x320 3 glottiss, 57.7ms
Speed: 0.8ms preprocess, 57.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 320)

0: 640x160 1 glottis, 60.0ms
Speed: 0.4ms preprocess, 60.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 160)

0: 640x160 1 glottis, 2.2ms
Speed: 0.3ms preprocess, 2.2ms inference, 0.9ms postprocess per image at shape 